# Connecting to Melbourne Data and Building an ETL

## Welcome to the Tutorial

In this tutorial you will learn how to connect to the City of Melbourne Data Portal using the API "sodapy".  Once connected you will download parking sensor data and build a rudimentary ETL (Extract Transform Load).  An ETL is what is used when collecting data automatically (Extract), doing something to the data (Transform) and putting the data somewhere to where it fits your needs (Load).  For our purposes we will download the data from the City of Melbourne Data Portal (Extract), adding the date and time to the data (Transform) and saving it to our local device (Load).  It is not as difficult as it sounds.  This tutorial is meant for beginners in Python and/or people who have never built an ETL before.

## Connecting to Melbourne Data

1.	Find Endpoint through website of the data set you need.
2.	Sign up at https://evergreen.data.socrata.com/signup/ to get a token so that you can call the data frequently (up to 1000 times per hour)
3.	Once signed up you can register for a token using https://support.socrata.com/hc/en-us/articles/210138558-Generating-an-App-Token to help guide you.
4.	Sign in to the Melbourne Data Portal using your Socrata login information.
5.	Click my profile > edit profile > developer settings
6.	Get an API Token (API KEY not needed for our purposes) to have direct communication with the Melbourne Data portal.

In [1]:
# found in the Parking Sensor API button
endpoint = "https://data.melbourne.vic.gov.au/resource/vh2v-4nfs.json"

In [2]:
#! pip install sodapy
#! pip install pandas

Install sodapy

120 seconds to finish the whole request


In [3]:
from sodapy import Socrata

client = Socrata(
    "data.melbourne.vic.gov.au",
    "EC65cHicC3xqFXHHv********", # app token, just used to reduce throttling. Have removed part of the token for Authentication Purposes
    timeout=120
)

Go and find the end point id.  It’s the part at the end of the web address that looks like random numbers and letters.

In [4]:
# set to large limit to include all available sensors
data = client.get("vh2v-4nfs", limit=200000)

## Building the ETL

In [5]:
#Put the data into a dataframe
import pandas as pd

df = pd.DataFrame(data) 

In [6]:
#Check the length
len(df)

1056

In [7]:
#Check the data and see what it looks like
df.head()

,bay_id,st_marker_id,status,location,lat,lon,:@computed_region_evbi_jbp8
0,6028,13067S,Present,"{'latitude': '-37.82135355038442', 'longitude'...",-37.82135355038442,144.94393155279823,1
1,5878,12904N,Present,"{'latitude': '-37.79832412543489', 'longitude'...",-37.79832412543489,144.96488078826815,1
2,5872,C9190,Unoccupied,"{'latitude': '-37.79847513739384', 'longitude'...",-37.79847513739384,144.96542559059571,1
3,632,91W,Present,"{'latitude': '-37.80883571544522', 'longitude'...",-37.80883571544522,144.97175471476578,1
4,5850,C9102,Unoccupied,"{'latitude': '-37.79838809150849', 'longitude'...",-37.79838809150849,144.96461950098268,1


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1056 entries, 0 to 1055
Data columns (total 7 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   bay_id                       1056 non-null   object
 1   st_marker_id                 1056 non-null   object
 2   status                       1056 non-null   object
 3   location                     1056 non-null   object
 4   lat                          1056 non-null   object
 5   lon                          1056 non-null   object
 6   :@computed_region_evbi_jbp8  1056 non-null   object
dtypes: object(7)
memory usage: 57.9+ KB


For the purposes of this ETL we only need 'bay_id' and 'status'. We will be adding 'hour', 'minute', 'dayofweek' and 'date'

In [216]:
df.drop(columns = ['st_marker_id', 'location', 'lat', 'lon', ':@computed_region_evbi_jbp8'], inplace = True)

In [217]:
df

,bay_id,status
0,5361,Unoccupied
1,5548,Unoccupied
2,5418,Unoccupied
3,5358,Unoccupied
4,1292,Unoccupied
...,...,...
1075,3719,Unoccupied
1076,3019,Present
1077,1471,Unoccupied
1078,3727,Unoccupied


Now add in 'hour', 'minute', 'dayofweek' and 'date'


In [218]:
import datetime

To do this we need to call specific values in the datetime package.  Specifically %M for minute, %H for Hour, %A for dayofweek

In [219]:
time = datetime.datetime.now().strftime
print(time("%M"))
print(time("%H"))
print(time("%A"))
date = datetime.date.today()
print(date)

39
21
Saturday
2021-08-07


In [220]:
#Add this all together

df['hour'] = time("%H")
df['minute'] = time("%M")
df['dayofweek'] = time("%A")
df['date'] = datetime.date.today()
df

,bay_id,status,hour,minute,dayofweek,date
0,5361,Unoccupied,21,39,Saturday,2021-08-07
1,5548,Unoccupied,21,39,Saturday,2021-08-07
2,5418,Unoccupied,21,39,Saturday,2021-08-07
3,5358,Unoccupied,21,39,Saturday,2021-08-07
4,1292,Unoccupied,21,39,Saturday,2021-08-07
...,...,...,...,...,...,...
1075,3719,Unoccupied,21,39,Saturday,2021-08-07
1076,3019,Present,21,39,Saturday,2021-08-07
1077,1471,Unoccupied,21,39,Saturday,2021-08-07
1078,3727,Unoccupied,21,39,Saturday,2021-08-07


For the purposes of this notebook and not the Python file, I will be showing you what the ETL will be doing on the server end.

In [206]:
#You may want to change the values.  See how df changes in length in the next cell
                    
i = 1
while i <= 2:
    df1 = pd.DataFrame(client.get("vh2v-4nfs", limit=200000))
    df1.drop(columns = ['st_marker_id', 'location', 'lat', 'lon', ':@computed_region_evbi_jbp8'], inplace = True)
    time = datetime.datetime.now().strftime
    df1['hour'] = time("%H")
    df1['minute'] = time("%M")
    df1['dayofweek'] = time("%A")
    df1['date'] = datetime.date.today()
    df = df.append(df1)
    i += 1

In [207]:
df

,bay_id,status,hour,minute,dayofweek,date
0,5361,Unoccupied,20,26,Saturday,2021-08-07
1,5548,Unoccupied,20,26,Saturday,2021-08-07
2,5418,Unoccupied,20,26,Saturday,2021-08-07
3,5358,Unoccupied,20,26,Saturday,2021-08-07
4,1292,Unoccupied,20,26,Saturday,2021-08-07
...,...,...,...,...,...,...
1075,3719,Unoccupied,20,26,Saturday,2021-08-07
1076,3019,Present,20,26,Saturday,2021-08-07
1077,1471,Unoccupied,20,26,Saturday,2021-08-07
1078,3727,Unoccupied,20,26,Saturday,2021-08-07


Now I will add the sleep function.  For starters lets see what sleep does first before adding into the loop

In [192]:
from time import sleep

print("Printed immediately.")
sleep(2*5)
print("Printed after 10 seconds.")

Printed immediately.
Printed after 10 seconds.


Okay lets run this to construct a dataset.  Be careful as this is meant to call the data many times and so the cell will run for a long time.  Feel free to skip the cell and check out the next one for how to save

In [ ]:
#BE CAREFUL WITH THIS CELL.  PLEASE CHANGE VALUES OF i AND sleep().  This is set to run for an hour at 5 minute intervals
from time import sleep

i = 1
while i <= 12:
    df1 = pd.DataFrame(client.get("vh2v-4nfs", limit=200000))
    df1.drop(columns = ['st_marker_id', 'location', 'lat', 'lon', ':@computed_region_evbi_jbp8'], inplace = True)
    time = datetime.datetime.now().strftime
    df1['hour'] = time("%H")
    df1['minute'] = time("%M")
    df1['dayofweek'] = time("%A")
    df1['date'] = datetime.date.today()
    df = df.append(df1)
    sleep(60*5)
    i += 1

In [209]:
#Save Data to CSV File
df.to_csv('OneHourData.csv')

In [222]:
#BE CAREFUL WITH THIS CELL.  PLEASE CHANGE VALUES OF i AND sleep().  This is set to run for 6 hours at 15 minute intervals
from time import sleep

i = 1
while i <= 24:
    df1 = pd.DataFrame(client.get("vh2v-4nfs", limit=200000))
    df1.drop(columns = ['st_marker_id', 'location', 'lat', 'lon', ':@computed_region_evbi_jbp8'], inplace = True)
    time = datetime.datetime.now().strftime
    df1['hour'] = time("%H")
    df1['minute'] = time("%M")
    df1['dayofweek'] = time("%A")
    df1['date'] = datetime.date.today()
    df = df.append(df1)
    sleep(60*15)
    i += 1

In [223]:
df.to_csv('SixHoursData.csv')

In [194]:
df

,bay_id,status,hour,minute,dayofweek,date
0,5361,Unoccupied,20,22,Saturday,2021-08-07
1,5548,Unoccupied,20,22,Saturday,2021-08-07
2,5418,Unoccupied,20,22,Saturday,2021-08-07
3,5358,Unoccupied,20,22,Saturday,2021-08-07
4,1292,Unoccupied,20,22,Saturday,2021-08-07
...,...,...,...,...,...,...
1075,3719,Unoccupied,20,25,Saturday,2021-08-07
1076,3019,Present,20,25,Saturday,2021-08-07
1077,1471,Unoccupied,20,25,Saturday,2021-08-07
1078,3727,Unoccupied,20,25,Saturday,2021-08-07


And that's it!  What you have created is a very simple ETL in a python notebook.  It is a little different in a cloud service like Amazon AWS or Microsoft Azure as you would not create a while loop but mention when the file has to be activated (in our case every 15 minutes) but the process is the same.  I hope you've enjoyed this tutorial and learnt the basics on how to create an ETL in pandas.

---

This notebook is created by Jason Tsitsopoulos.  You can reach me at jzt750 [at] gmail.com or check out my git https://github.com/JasonT2021